# CCS
preprocessing for CCS prediction

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import re, os, csv
import pickle

data_path = 'data/combined.csv'
outpath = 'data_final/Tests/200206_ward_min2_PTtest/'

In [12]:
data6 = pd.read_csv(data_path)
data6 = data6.rename(index=str, columns={"Modified sequence": "Modified_sequence"})
data6['Modified_sequence'] = data6['Modified_sequence'].str.replace('_','')
print('read data')

read data


In [13]:
def remove10(data):
    #remove upper/lower 10 percent
    print(data.shape)
    low = np.percentile(data['Sum_intensity'], 10)
    high = np.percentile(data['Sum_intensity'], 90)
    print(low, high)
    d=data
    d = d[d['Sum_intensity'] > low]
    #d = d[d['Sum_intensity'] < high]
    print(d.shape)
    #d6 = d6.groupby(['Modified_sequence', 'Charge'], group_keys=False).apply(lambda x: x.loc[x.Intensity.idxmax()])
    return d

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)


def split(data, name, s, label_encoder_path='data_final/enc.pickle', ids=None, calc_minval=True):
    ensure_dir(name)
    np.random.seed(s)
    with open(label_encoder_path, 'rb') as handle:
        label_encoder = pickle.load(handle)
    data['encseq'] = data['Modified_sequence'].apply(lambda x: label_encoder.transform(list(x)))
    if calc_minval:
        data['minval'] = np.min(data['label'])
        data['maxval'] = np.max(data['label'])
    else:
        data['minval']=275.440277
        data['maxval']=1112.030762
        
    if ids == None:
        #a = np.random.uniform(0.0,1.0,len(data)) > 0.98
        data['test'] = data['PT']
        print('Using proteome tools testsset')
        
    else:
        print('using predefined testset')
        data['test'] = ~data['Modified_sequence'].isin(ids)

    data['task'] = 0
    print('Name: ', name, 'Seed: ', s, 'Len test: ', len(data[data['test']]),'Len set test: ', len(set(data[data['test']])),'Len not test: ', len(data[~data['test']]),'Len set not test: ', len(set(data[~data['test']])))
    data[~data['test']].to_pickle(name + str(s) + '_train.pkl')
    data[data['test']].to_pickle(name +str(s) + '_test.pkl')
    return data
def kill_new_seqs(df):
    df['label']=np.zeros(len(df))
    print(len(df), 'total seqs')
    seqs = df['Modified_sequence'].values.tolist()
    flat_list = [item for sublist in seqs for item in sublist]
    s = set(flat_list)
    with open('data_final/enc_list.csv') as f:
        reader = csv.reader(f)
        enc_list = list(reader)[0]
        print(enc_list)
    new_acids = list(set(s)-set(enc_list))
    if len(new_acids) != 0:
        df_red = df[~df['Modified_sequence'].str.contains('|'.join(new_acids))]
    else:
        df_red = df
    print(len(df_red), 'reduced seqs', new_acids)
    return df_red

In [14]:
data6.head()

,Unnamed: 0,Modified_sequence,Sequence,Charge,Mass,m/z,Experiment,id,Intensity,Score,Length,Retention time,CCS,PT
0,830179,(ac)AAAAAAAAAAGAAGGR,AAAAAAAAAAGAAGGR,2,1239.63200,620.823275,HeLa_Trp_2,9,149810.0,162.800,16,70.140,409.092529,False
1,11,(ac)AAAAAAAAEQQSSNGPVKK,AAAAAAAAEQQSSNGPVKK,2,1810.91734,906.465946,Proteotypic,11,21349.0,89.382,19,19.645,481.229248,True
2,2044072,(ac)AAAAAAAGAAGSAAPAAAAGAPGSGGAPSGSQGVLIGDR,AAAAAAAGAAGSAAPAAAAGAPGSGGAPSGSQGVLIGDR,3,3144.55482,1049.192220,HeLa_Trypsin_1,124,194000.0,71.438,39,3947.700,772.098083,False
3,830196,(ac)AAAAAAAGDSDSWDADAFSVEDPVRK,AAAAAAAGDSDSWDADAFSVEDPVRK,2,2634.18340,1318.098980,HeLa_Trp_2,26,6416400.0,111.750,26,94.079,573.213196,False
4,830200,(ac)AAAAAAAGDSDSWDADAFSVEDPVRK,AAAAAAAGDSDSWDADAFSVEDPVRK,3,2634.18340,879.068411,HeLa_Trp_2,30,5400600.0,42.218,26,94.841,635.000549,False


# Create Sequence one hot encoder

In [15]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

data=data6
dat = data['Modified_sequence']
dat = [list(d) for d in dat]    

#process data into one hot encoding
flat_list = ['_'] + [item for sublist in dat for item in sublist]

# define example
values = np.array(flat_list)

label_encoder = LabelEncoder()
label_encoder.fit(values)

print(label_encoder.classes_, len(label_encoder.classes_))
with open('data_final/enc.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle)
import csv
with open('data_final/enc_list.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(list(label_encoder.classes_))
    
    
    

['(' ')' 'A' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'K' 'L' 'M' 'N' 'P' 'Q' 'R' 'S'
 'T' 'V' 'W' 'Y' '_' 'a' 'c' 'o' 'x'] 27


In [16]:
print(label_encoder.transform([['_']]), 22)


[22] 22


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Convert CSS training data

In [7]:
data6.shape

(1057682, 4)

In [10]:
data6 = data6[~data6['Intensity'].isnull()]
data6 = data6[~data6['CCS'].isnull()]
#data6['Modified_sequence'] = data6['Modified_sequence'].str.replace('_','')

# for multiple with same sequence and charge take the one with highest intensity
#d6 = data6.groupby(['Modified_sequence', 'Charge'], group_keys=False).apply(lambda x: x.loc[x.Intensity.idxmax()])
d6 = data6
print('done grouping')
d6['label']=d6['CCS'].values.tolist()
dd = split(d6, outpath, 2)

done grouping


KeyError: 'CCS'

In [ ]:
trainseqs = dd[~dd['test']]['Modified_sequence'].values.tolist()
ddd = dd[dd['test']]
ddd[ddd['Modified_sequence'].isin(trainseqs)].shape

In [ ]:
dd = split(d6, outpath, 2, ids=trainseqs)

In [ ]:
trainseqs = dd[~dd['test']]['Modified_sequence'].values.tolist()
ddd = dd[dd['test']]
ddd[ddd['Modified_sequence'].isin(trainseqs)].shape

In [ ]:
len(ddd)